In [0]:
import copy
import yaml
import argparse
import optuna
import mlflow
from functools import partial

from main import run_experiment

In [0]:
def objective(trial, base_config, model_name):
    config = copy.deepcopy(base_config)  # Ikke endre originalen

    if model_name == "unet":
        config["model"]["unet"]["lr"] = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
        config["data"]["batch_size"] = trial.suggest_categorical(
            "batch_size", [4, 8, 16]
        )

    trial.set_user_attr("lr", config["model"][model_name]["lr"])
    trial.set_user_attr("batch_size", config["data"]["batch_size"])

    val_metric = run_experiment(model_name, config)

    # Logger til Optuna
    trial.report(val_metric, step=0)

    return val_metric

In [0]:
def main(config_path):
    # Load config YAML
    with open(config_path) as f:
        config = yaml.safe_load(f)

    model_name = config["model_names"][0]  # Tune one at a time

    direction = config["optuna"]["direction"]
    n_trials = config["optuna"]["n_trials"]
    n_jobs = config["optuna"].get("n_jobs", 1)  # Optional parallelism config

    study = optuna.create_study(direction="maximize")
    
    # Optimize in parallel (if n_jobs > 1)
    study.optimize(
        lambda trial: objective(trial, config, model_name),
        n_trials=n_trials,
        n_jobs=n_jobs,
    )

    # Log best parameters and score
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_val_accuracy", study.best_value)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--config", type=str, default="static.yaml", help="Path to YAML config"
    )

    args, unknown = parser.parse_known_args()

    print(unknown)
    print(args.config)

    main(args.config)